In [2]:
!pip install pysrt

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'pysrt' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pysrt'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13502 sha256=5751ef23e7cb4991f95b23efa7d775acc339e7d1821be1c5619dc4502ae291b4
  Stored in directory: /home/dev/.cache/pip/wheels/1c/af/06/094acdd1f5840fd0b55bb1c728924f3c3dacbf3fa59fb38a0b
Successfully built pysrt
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pysrt]t]


In [ ]:
import cv2
import numpy as np
from collections import defaultdict, deque
import matplotlib.pyplot as plt
import pysrt
import folium
from folium import plugins
import re
from scipy.spatial import distance
from scipy.optimize import linear_sum_assignment


class EnhancedCarTracker:
    
    def __init__(self, video_path, srt_path=None, config=None):
        self.video_path = video_path
        self.srt_path = srt_path
        self.cap = cv2.VideoCapture(video_path)
        
        if not self.cap.isOpened():
            raise ValueError(f"Cannot open video file: {video_path}")
        
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        print(f"Video info: {self.width}x{self.height}, {self.fps:.2f} FPS, {self.total_frames} frames")
        
        self.tracks = defaultdict(list)
        self.next_car_id = 0
        self.active_tracks = {}
        
        self.config = {
            'max_distance': 80,
            'min_area': 400,
            'max_area': 50000,
            'min_aspect_ratio': 0.4,
            'max_aspect_ratio': 4.0,
            'min_track_length': 15,
            'frame_skip': 2,
            'history_length': 5,
            'scale_factor': 0.5,
        }
        
        if config:
            self.config.update(config)
        
        if srt_path:
            self.srt_data = self.load_srt()
        else:
            self.srt_data = []
    
    def load_srt(self):
        try:
            subs = pysrt.open(self.srt_path)
            gps_data = []
            
            for sub in subs:
                text = sub.text
                
                lat_match = re.search(r'\[latitude:\s*([-\d.]+)\]', text)
                lon_match = re.search(r'\[longitude:\s*([-\d.]+)\]', text)
                
                if lat_match and lon_match:
                    try:
                        lat = float(lat_match.group(1))
                        lon = float(lon_match.group(1))
                        
                        timestamp = (sub.start.hours * 3600 + 
                                   sub.start.minutes * 60 + 
                                   sub.start.seconds + 
                                   sub.start.milliseconds / 1000.0)
                        
                        gps_data.append({
                            'time': timestamp,
                            'lat': lat,
                            'lon': lon,
                            'frame': len(gps_data) + 1
                        })
                    except (ValueError, AttributeError) as e:
                        continue
            
            print(f"✓ Loaded {len(gps_data)} GPS points from SRT file")
            return gps_data
            
        except Exception as e:
            print(f"✗ Error loading SRT file: {e}")
            return []
    
    def get_gps_at_time(self, timestamp):
        if not self.srt_data or len(self.srt_data) == 0:
            return None, None
        
        for i in range(len(self.srt_data) - 1):
            if self.srt_data[i]['time'] <= timestamp <= self.srt_data[i+1]['time']:
                t1 = self.srt_data[i]['time']
                t2 = self.srt_data[i+1]['time']
                
                if t2 == t1:
                    ratio = 0
                else:
                    ratio = (timestamp - t1) / (t2 - t1)
                
                lat1, lon1 = self.srt_data[i]['lat'], self.srt_data[i]['lon']
                lat2, lon2 = self.srt_data[i+1]['lat'], self.srt_data[i+1]['lon']
                
                lat = lat1 + ratio * (lat2 - lat1)
                lon = lon1 + ratio * (lon2 - lon1)
                
                return lat, lon
        
        if timestamp < self.srt_data[0]['time']:
            return self.srt_data[0]['lat'], self.srt_data[0]['lon']
        else:
            return self.srt_data[-1]['lat'], self.srt_data[-1]['lon']
    
    def detect_cars_in_frame(self, frame, back_sub):
        scale = self.config['scale_factor']
        small_frame = cv2.resize(frame, None, fx=scale, fy=scale)
        
        fg_mask = back_sub.apply(small_frame)
        
        _, fg_mask = cv2.threshold(fg_mask, 250, 255, cv2.THRESH_BINARY)
        
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel, iterations=1)
        
        contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        detections = []
        
        for contour in contours:
            area = cv2.contourArea(contour)
            
            if self.config['min_area'] < area < self.config['max_area']:
                x, y, w, h = cv2.boundingRect(contour)
                
                aspect_ratio = w / h if h > 0 else 0
                if (self.config['min_aspect_ratio'] < aspect_ratio < 
                    self.config['max_aspect_ratio']):
                    
                    center_x = int((x + w / 2) / scale)
                    center_y = int((y + h / 2) / scale)
                    orig_w = int(w / scale)
                    orig_h = int(h / scale)
                    
                    detections.append((center_x, center_y, orig_w, orig_h, area))
        
        return detections
    
    def match_detections_to_tracks(self, detections, timestamp):
        if len(detections) == 0:
            return {}
        
        if len(self.active_tracks) == 0:
            matched = {}
            for det in detections:
                matched[self.next_car_id] = {
                    'centroid': (det[0], det[1]),
                    'width': det[2],
                    'height': det[3],
                    'area': det[4],
                    'timestamp': timestamp
                }
                self.next_car_id += 1
            return matched
        
        track_ids = list(self.active_tracks.keys())
        cost_matrix = np.zeros((len(track_ids), len(detections)))
        
        for i, track_id in enumerate(track_ids):
            track = self.active_tracks[track_id]
            last_pos = track['centroids'][-1]
            
            for j, det in enumerate(detections):
                det_pos = (det[0], det[1])
                dist = np.sqrt((last_pos[0] - det_pos[0])**2 + 
                             (last_pos[1] - det_pos[1])**2)
                cost_matrix[i, j] = dist
        
        if cost_matrix.size > 0:
            row_ind, col_ind = linear_sum_assignment(cost_matrix)
        else:
            row_ind, col_ind = [], []
        
        matched = {}
        max_dist = self.config['max_distance']
        
        matched_detection_indices = set()
        for i, j in zip(row_ind, col_ind):
            if cost_matrix[i, j] < max_dist:
                track_id = track_ids[i]
                det = detections[j]
                matched[track_id] = {
                    'centroid': (det[0], det[1]),
                    'width': det[2],
                    'height': det[3],
                    'area': det[4],
                    'timestamp': timestamp
                }
                matched_detection_indices.add(j)
        
        for j, det in enumerate(detections):
            if j not in matched_detection_indices:
                matched[self.next_car_id] = {
                    'centroid': (det[0], det[1]),
                    'width': det[2],
                    'height': det[3],
                    'area': det[4],
                    'timestamp': timestamp
                }
                self.next_car_id += 1
        
        return matched
    
    def update_tracks(self, matched_detections):
        new_active_tracks = {}
        
        for track_id, det_data in matched_detections.items():
            if track_id in self.active_tracks:
                track = self.active_tracks[track_id]
            else:
                track = {
                    'centroids': deque(maxlen=self.config['history_length']),
                    'timestamps': deque(maxlen=self.config['history_length']),
                    'widths': deque(maxlen=self.config['history_length']),
                    'heights': deque(maxlen=self.config['history_length']),
                    'first_seen': det_data['timestamp']
                }
            
            track['centroids'].append(det_data['centroid'])
            track['timestamps'].append(det_data['timestamp'])
            track['widths'].append(det_data['width'])
            track['heights'].append(det_data['height'])
            track['last_seen'] = det_data['timestamp']
            
            new_active_tracks[track_id] = track
        
        self.active_tracks = new_active_tracks
    
    def save_track_point(self, track_id, frame_num, timestamp):
        if track_id not in self.active_tracks:
            return
        
        track = self.active_tracks[track_id]
        centroid = track['centroids'][-1]
        
        lat, lon = self.get_gps_at_time(timestamp)
        
        self.tracks[track_id].append({
            'frame': frame_num,
            'timestamp': timestamp,
            'pixel_x': centroid[0],
            'pixel_y': centroid[1],
            'width': track['widths'][-1],
            'height': track['heights'][-1],
            'lat': lat,
            'lon': lon
        })
    
    def detect_and_track_cars(self, save_annotated_video=False):
        back_sub = cv2.createBackgroundSubtractorMOG2(
            history=500, 
            varThreshold=50, 
            detectShadows=True
        )
        
        frame_count = 0
        processed_count = 0
        
        print("\n" + "="*60)
        print("Starting car detection and tracking...")
        print("="*60)
        
        video_writer = None
        if save_annotated_video:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            video_writer = cv2.VideoWriter(
                'tracked_output.mp4',
                fourcc,
                self.fps,
                (self.width, self.height)
            )
        
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break
            
            frame_count += 1
            timestamp = frame_count / self.fps
            
            if frame_count % self.config['frame_skip'] != 0:
                continue
            
            processed_count += 1
            
            detections = self.detect_cars_in_frame(frame, back_sub)
            
            matched = self.match_detections_to_tracks(detections, timestamp)
            
            self.update_tracks(matched)
            
            for track_id in self.active_tracks.keys():
                self.save_track_point(track_id, frame_count, timestamp)
            
            if save_annotated_video and video_writer:
                annotated_frame = frame.copy()
                for track_id, track in self.active_tracks.items():
                    centroid = track['centroids'][-1]
                    cv2.circle(annotated_frame, centroid, 5, (0, 255, 0), -1)
                    cv2.putText(annotated_frame, f"ID:{track_id}", 
                              (centroid[0] + 10, centroid[1] - 10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
                    if len(track['centroids']) > 1:
                        points = np.array(list(track['centroids']), dtype=np.int32)
                        cv2.polylines(annotated_frame, [points], False, (255, 0, 0), 2)
                
                video_writer.write(annotated_frame)
            
            if processed_count % 100 == 0:
                progress = (frame_count / self.total_frames) * 100
                print(f"Progress: {progress:.1f}% | Frame: {frame_count}/{self.total_frames} | "
                      f"Active tracks: {len(self.active_tracks)} | "
                      f"Total tracks: {len(self.tracks)}")
        
        self.cap.release()
        if video_writer:
            video_writer.release()
        
        print("\n" + "="*60)
        print(f"Tracking complete!")
        print(f"Total frames processed: {processed_count}")
        print(f"Total tracks before filtering: {len(self.tracks)}")
        print("="*60)
        
        min_length = self.config['min_track_length']
        filtered_tracks = {
            k: v for k, v in self.tracks.items() 
            if len(v) >= min_length
        }
        
        print(f"\nAfter filtering (min {min_length} frames): {len(filtered_tracks)} valid tracks")
        self.tracks = filtered_tracks
        
        return self.tracks
    
    def plot_paths_static(self, output_path='car_paths_visualization.png'):
        if len(self.tracks) == 0:
            print("No tracks to plot!")
            return None
        
        fig = plt.figure(figsize=(20, 10))
        
        gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)
        ax1 = fig.add_subplot(gs[0, 0])
        ax2 = fig.add_subplot(gs[0, 1])
        ax3 = fig.add_subplot(gs[1, :])
        
        colors = plt.cm.rainbow(np.linspace(0, 1, len(self.tracks)))
        
        ax1.set_title('Car Paths in Video Space', fontsize=14, fontweight='bold')
        ax1.set_xlabel('X Position (pixels)', fontsize=11)
        ax1.set_ylabel('Y Position (pixels)', fontsize=11)
        ax1.invert_yaxis()
        ax1.set_facecolor('#f0f0f0')
        
        ax2.set_title('Car Paths in GPS Space', fontsize=14, fontweight='bold')
        ax2.set_xlabel('Longitude', fontsize=11)
        ax2.set_ylabel('Latitude', fontsize=11)
        ax2.set_facecolor('#f0f0f0')
        
        ax3.set_title('Car Detection Timeline', fontsize=14, fontweight='bold')
        ax3.set_xlabel('Time (seconds)', fontsize=11)
        ax3.set_ylabel('Car ID', fontsize=11)
        ax3.set_facecolor('#f0f0f0')
        
        has_gps = False
        
        for idx, (car_id, path) in enumerate(sorted(self.tracks.items())):
            if len(path) < 2:
                continue
            
            color = colors[idx]
            
            x_coords = [p['pixel_x'] for p in path]
            y_coords = [p['pixel_y'] for p in path]
            times = [p['timestamp'] for p in path]
            
            ax1.plot(x_coords, y_coords, '-', color=color, 
                    linewidth=2, alpha=0.7, label=f'Car {car_id}')
            ax1.scatter(x_coords[0], y_coords[0], color=color, 
                       s=150, marker='o', edgecolors='black', linewidths=2, zorder=5)
            ax1.scatter(x_coords[-1], y_coords[-1], color=color, 
                       s=150, marker='s', edgecolors='black', linewidths=2, zorder=5)
            
            gps_path = [(p['lat'], p['lon']) for p in path 
                       if p['lat'] is not None and p['lon'] is not None]
            
            if len(gps_path) >= 2:
                has_gps = True
                lats = [p[0] for p in gps_path]
                lons = [p[1] for p in gps_path]
                
                ax2.plot(lons, lats, '-', color=color, 
                        linewidth=2, alpha=0.7, label=f'Car {car_id}')
                ax2.scatter(lons[0], lats[0], color=color, 
                           s=150, marker='o', edgecolors='black', linewidths=2, zorder=5)
                ax2.scatter(lons[-1], lats[-1], color=color, 
                           s=150, marker='s', edgecolors='black', linewidths=2, zorder=5)
            
            ax3.barh(car_id, times[-1] - times[0], left=times[0], 
                    height=0.8, color=color, alpha=0.7, edgecolor='black')
            ax3.text(times[0] + (times[-1] - times[0]) / 2, car_id, 
                    f'{len(path)} pts', ha='center', va='center', fontsize=8)
        
        if len(self.tracks) <= 20:
            ax1.legend(loc='best', fontsize=8, ncol=2)
        ax1.grid(True, alpha=0.3)
        
        if has_gps:
            if len(self.tracks) <= 20:
                ax2.legend(loc='best', fontsize=8, ncol=2)
            ax2.grid(True, alpha=0.3)
        else:
            ax2.text(0.5, 0.5, 'No GPS data available', 
                    ha='center', va='center', transform=ax2.transAxes, 
                    fontsize=14, color='red')
        
        ax3.grid(True, alpha=0.3, axis='x')
        ax3.set_yticks(sorted(self.tracks.keys()))
        
        total_duration = max([p[-1]['timestamp'] for p in self.tracks.values()])
        fig.suptitle(f'Car Tracking Results: {len(self.tracks)} cars tracked over {total_duration:.1f} seconds', 
                    fontsize=16, fontweight='bold')
        
        plt.savefig(output_path, dpi=200, bbox_inches='tight')
        print(f"✓ Static visualization saved to: {output_path}")
        plt.close()
        
        return output_path
    
    def create_interactive_map(self, output_path='car_paths_interactive.html'):
        if len(self.tracks) == 0:
            print("No tracks to plot!")
            return None
        
        has_gps = False
        all_lats, all_lons = [], []
        
        for path in self.tracks.values():
            for p in path:
                if p['lat'] is not None and p['lon'] is not None:
                    has_gps = True
                    all_lats.append(p['lat'])
                    all_lons.append(p['lon'])
        
        if not has_gps:
            print("✗ No GPS data available for interactive map!")
            return None
        
        center_lat = np.mean(all_lats)
        center_lon = np.mean(all_lons)
        
        print(f"Creating interactive map centered at: {center_lat:.6f}, {center_lon:.6f}")
        
        m = folium.Map(
            location=[center_lat, center_lon],
            zoom_start=16,
            tiles='OpenStreetMap'
        )
        
        folium.TileLayer('CartoDB positron', name='Light Map').add_to(m)
        folium.TileLayer('CartoDB dark_matter', name='Dark Map').add_to(m)
        
        folium.TileLayer(
            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr='Esri',
            name='Satellite',
            overlay=False,
            control=True
        ).add_to(m)
        
        colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 
                  'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 
                  'darkpurple', 'pink', 'lightblue', 'lightgreen', 'gray', 
                  'black', 'lightgray', 'white', 'darkgray']
        
        for idx, (car_id, path) in enumerate(sorted(self.tracks.items())):
            if len(path) < 2:
                continue
            
            gps_path = [(p['lat'], p['lon']) for p in path 
                       if p['lat'] is not None and p['lon'] is not None]
            
            if len(gps_path) < 2:
                continue
            
            color = colors[idx % len(colors)]
            
            duration = path[-1]['timestamp'] - path[0]['timestamp']
            num_points = len(path)
            
            feature_group = folium.FeatureGroup(
                name=f'Car {car_id} ({num_points} points, {duration:.1f}s)',
                show=True
            )
            
            folium.PolyLine(
                gps_path,
                color=color,
                weight=4,
                opacity=0.8,
                popup=f'<b>Car {car_id}</b><br>Points: {num_points}<br>Duration: {duration:.1f}s',
                tooltip=f'Car {car_id}'
            ).add_to(feature_group)
            
            folium.CircleMarker(
                location=gps_path[0],
                radius=10,
                popup=f'<b>Car {car_id} - START</b><br>Time: {path[0]["timestamp"]:.1f}s',
                tooltip=f'Car {car_id} START',
                color=color,
                fill=True,
                fillColor='white',
                fillOpacity=0.9,
                weight=4
            ).add_to(feature_group)
            
            folium.Marker(
                location=gps_path[-1],
                popup=f'<b>Car {car_id} - END</b><br>Time: {path[-1]["timestamp"]:.1f}s<br>Duration: {duration:.1f}s',
                tooltip=f'Car {car_id} END',
                icon=folium.Icon(color=color, icon='stop', prefix='fa')
            ).add_to(feature_group)
            
            for i in range(0, len(path), 10):
                if path[i]['lat'] is not None and path[i]['lon'] is not None:
                    folium.CircleMarker(
                        location=(path[i]['lat'], path[i]['lon']),
                        radius=3,
                        color=color,
                        fill=True,
                        fillColor=color,
                        fillOpacity=0.6,
                        weight=1,
                        popup=f'Car {car_id}<br>Time: {path[i]["timestamp"]:.1f}s<br>Frame: {path[i]["frame"]}'
                    ).add_to(feature_group)
            
            feature_group.add_to(m)
        
        folium.LayerControl(collapsed=False).add_to(m)
        plugins.Fullscreen(position='topright').add_to(m)
        plugins.MeasureControl(position='topleft', primary_length_unit='meters').add_to(m)
        plugins.MiniMap(toggle_display=True).add_to(m)
        plugins.MousePosition().add_to(m)
        
        plugins.Draw(export=True).add_to(m)
        
        m.save(output_path)
        print(f"✓ Interactive map saved to: {output_path}")
        
        return output_path
    
    def generate_statistics_report(self, output_path='tracking_report.txt'):
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write("="*70 + "\n")
            f.write("CAR TRACKING STATISTICS REPORT\n")
            f.write("="*70 + "\n\n")
            
            f.write(f"Video: {self.video_path}\n")
            f.write(f"Resolution: {self.width}x{self.height}\n")
            f.write(f"FPS: {self.fps:.2f}\n")
            f.write(f"Total Frames: {self.total_frames}\n\n")
            
            f.write(f"Total Cars Tracked: {len(self.tracks)}\n")
            f.write(f"GPS Data Available: {'Yes' if self.srt_data else 'No'}\n")
            if self.srt_data:
                f.write(f"GPS Points: {len(self.srt_data)}\n")
            f.write("\n" + "-"*70 + "\n\n")
            
            f.write("INDIVIDUAL CAR STATISTICS:\n\n")
            
            for car_id, path in sorted(self.tracks.items()):
                f.write(f"Car ID {car_id}:\n")
                f.write(f"  • Total Points: {len(path)}\n")
                f.write(f"  • First Seen: Frame {path[0]['frame']}, Time {path[0]['timestamp']:.2f}s\n")
                f.write(f"  • Last Seen: Frame {path[-1]['frame']}, Time {path[-1]['timestamp']:.2f}s\n")
                
                duration = path[-1]['timestamp'] - path[0]['timestamp']
                f.write(f"  • Duration: {duration:.2f} seconds\n")
                
                total_pixel_dist = 0
                for i in range(1, len(path)):
                    dx = path[i]['pixel_x'] - path[i-1]['pixel_x']
                    dy = path[i]['pixel_y'] - path[i-1]['pixel_y']
                    total_pixel_dist += np.sqrt(dx**2 + dy**2)
                
                f.write(f"  • Pixel Distance Traveled: {total_pixel_dist:.2f} pixels\n")
                
                if path[0]['lat'] is not None:
                    gps_dist = self.calculate_gps_distance(path)
                    if gps_dist > 0:
                        f.write(f"  • GPS Distance Traveled: {gps_dist:.2f} meters\n")
                        avg_speed = (gps_dist / duration) if duration > 0 else 0
                        f.write(f"  • Average Speed: {avg_speed:.2f} m/s ({avg_speed*3.6:.2f} km/h)\n")
                
                f.write("\n")
            
            f.write("-"*70 + "\n\n")
            
            f.write("SUMMARY STATISTICS:\n\n")
            all_durations = [p[-1]['timestamp'] - p[0]['timestamp'] for p in self.tracks.values()]
            all_lengths = [len(p) for p in self.tracks.values()]
            
            f.write(f"  • Average Track Duration: {np.mean(all_durations):.2f}s\n")
            f.write(f"  • Median Track Duration: {np.median(all_durations):.2f}s\n")
            f.write(f"  • Average Points per Track: {np.mean(all_lengths):.1f}\n")
            f.write(f"  • Median Points per Track: {np.median(all_lengths):.1f}\n")
            f.write(f"  • Shortest Track: {min(all_durations):.2f}s ({min(all_lengths)} points)\n")
            f.write(f"  • Longest Track: {max(all_durations):.2f}s ({max(all_lengths)} points)\n")
            
            f.write("\n" + "="*70 + "\n")
        
        print(f"✓ Statistics report saved to: {output_path}")
        return output_path
    
    def calculate_gps_distance(self, path):
        total_dist = 0
        
        for i in range(1, len(path)):
            if (path[i-1]['lat'] is not None and path[i-1]['lon'] is not None and
                path[i]['lat'] is not None and path[i]['lon'] is not None):
                
                lat1, lon1 = np.radians(path[i-1]['lat']), np.radians(path[i-1]['lon'])
                lat2, lon2 = np.radians(path[i]['lat']), np.radians(path[i]['lon'])
                
                dlat = lat2 - lat1
                dlon = lon2 - lon1
                
                a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
                c = 2 * np.arcsin(np.sqrt(a))
                
                r = 6371000
                total_dist += c * r
        
        return total_dist


def main():
    import sys
    
    print("\n" + "="*70)
    print(" ENHANCED CAR TRACKING SYSTEM ".center(70))
    print("="*70 + "\n")
    
    video_path = "video2.mp4"
    srt_path = "video2.srt"
    
    import os
    if not os.path.exists(video_path):
        print(f"✗ Error: Video file not found: {video_path}")
        return
    
    if not os.path.exists(srt_path):
        print(f"⚠ Warning: SRT file not found: {srt_path}")
        srt_path = None
    
    config = {
        'max_distance': 80,
        'min_area': 400,
        'max_area': 50000,
        'min_aspect_ratio': 0.4,
        'max_aspect_ratio': 4.0,
        'min_track_length': 15,
        'frame_skip': 2,
        'scale_factor': 0.5,
    }
    
    print("Initializing tracker...")
    tracker = EnhancedCarTracker(video_path, srt_path, config)
    
    tracks = tracker.detect_and_track_cars(save_annotated_video=False)
    
    if len(tracks) > 0:
        print("\n" + "="*70)
        print("Generating visualizations and reports...")
        print("="*70 + "\n")
        
        tracker.plot_paths_static('car_paths_visualization.png')
        tracker.create_interactive_map('car_paths_interactive.html')
        tracker.generate_statistics_report('tracking_report.txt')
        
        print("\n" + "="*70)
        print(" TRACKING COMPLETE! ".center(70))
        print("="*70)
        print("\nGenerated files:")
        print("  📊 car_paths_visualization.png - Static visualization")
        print("  🗺️  car_paths_interactive.html - Interactive map")
        print("  📋 tracking_report.txt - Statistics report")
        print("\n")
    else:
        print("\n✗ No cars were detected. Try adjusting detection parameters.")


if __name__ == "__main__":
    main()


                     ENHANCED CAR TRACKING SYSTEM                     

Initializing tracker...
Video info: 1920x1080, 29.97 FPS, 4979 frames
✓ Loaded 4979 GPS points from SRT file

Starting car detection and tracking...
Progress: 4.0% | Frame: 200/4979 | Active tracks: 5 | Total tracks: 170
Progress: 8.0% | Frame: 400/4979 | Active tracks: 9 | Total tracks: 855
Progress: 12.1% | Frame: 600/4979 | Active tracks: 1 | Total tracks: 1023
Progress: 16.1% | Frame: 800/4979 | Active tracks: 5 | Total tracks: 1174
Progress: 20.1% | Frame: 1000/4979 | Active tracks: 1 | Total tracks: 1385
Progress: 24.1% | Frame: 1200/4979 | Active tracks: 9 | Total tracks: 1575
Progress: 28.1% | Frame: 1400/4979 | Active tracks: 2 | Total tracks: 1856
Progress: 32.1% | Frame: 1600/4979 | Active tracks: 10 | Total tracks: 2094
Progress: 36.2% | Frame: 1800/4979 | Active tracks: 13 | Total tracks: 2635
Progress: 40.2% | Frame: 2000/4979 | Active tracks: 8 | Total tracks: 3231
Progress: 44.2% | Frame: 2200/4979

/tmp/ipykernel_213725/157999504.py:694: UserWarning: color argument of Icon should be one of: {'pink', 'orange', 'lightgreen', 'darkred', 'black', 'gray', 'lightred', 'lightgray', 'cadetblue', 'lightblue', 'blue', 'darkblue', 'beige', 'darkgreen', 'purple', 'red', 'white', 'green', 'darkpurple'}.
  icon=folium.Icon(color=color, icon='stop', prefix='fa')


✓ Interactive map saved to: car_paths_interactive.html
✓ Statistics report saved to: tracking_report.txt

                          TRACKING COMPLETE!                          

Generated files:
  📊 car_paths_visualization.png - Static visualization
  🗺️  car_paths_interactive.html - Interactive map
  📋 tracking_report.txt - Statistics report


